In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls gdrive/MyDrive/
!mkdir crops
!tar xvzf gdrive/MyDrive/CameraPrIMuS.tgz  -C crops --skip-old-files
!ls gdrive/MyDrive/

串流輸出內容已截斷至最後 5000 行。
Corpus/000103451-1_2_1/regular_pae.pae
Corpus/000103451-1_2_1/000103451-1_2_1.agnostic
Corpus/000103451-1_2_1/000103451-1_2_1_distorted.jpg
Corpus/000103451-1_2_1/000103451-1_2_1.semantic
Corpus/000103451-1_2_1/000103451-1_2_1.mei
Corpus/000103451-1_2_1/000103451-1_2_1.png
Corpus/000104921-1_10_1/
Corpus/000104921-1_10_1/regular_pae.pae
Corpus/000104921-1_10_1/000104921-1_10_1.png
Corpus/000104921-1_10_1/000104921-1_10_1.agnostic
Corpus/000104921-1_10_1/000104921-1_10_1_distorted.jpg
Corpus/000104921-1_10_1/000104921-1_10_1.semantic
Corpus/000104921-1_10_1/000104921-1_10_1.mei
Corpus/000102440-1_1_1/
Corpus/000102440-1_1_1/regular_pae.pae
Corpus/000102440-1_1_1/000102440-1_1_1.png
Corpus/000102440-1_1_1/000102440-1_1_1.agnostic
Corpus/000102440-1_1_1/000102440-1_1_1.semantic
Corpus/000102440-1_1_1/000102440-1_1_1_distorted.jpg
Corpus/000102440-1_1_1/000102440-1_1_1.mei
Corpus/201004360-1_1_2/
Corpus/201004360-1_1_2/regular_pae.pae
Corpus/201004360-1_1_2/201004360-1

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

import ctc_utils
import ctc_model
import argparse
from tensorflow.python.framework import ops
import os
from primus import CTC_PriMuS

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.reset_default_graph()
sess = tf.InteractiveSession(config=config)

class hi():
  def __init__(self):
    self.semantic=True
    self.corpus='/content/crops/Corpus'
    self.set = '/content/train.txt'
    self.voc='/content/vocabulary_semantic.txt'
    self.save_model='./trained_semantic_model'

args=hi();

# Load primus

primus = CTC_PriMuS(args.corpus,args.set,args.voc, args.semantic, val_split = 0.1)

# Parameterization
img_height = 128
print(ctc_model)
params = ctc_model.default_model_params(img_height,primus.vocabulary_size)
max_epochs = 64000
dropout = 0.5

# Model
inputs, seq_len, targets, decoded, loss, rnn_keep_prob = ctc_model.ctc_crnn(params)
train_opt = tf.train.AdamOptimizer().minimize(loss)


saver = tf.train.Saver(max_to_keep=None)
sess.run(tf.global_variables_initializer())
saver = tf.train.import_meta_graph("trained_semantic_model-7800.meta")
saver.restore(sess,"trained_semantic_model-7800")

# Training loop
for epoch in range(max_epochs):
    batch = primus.nextBatch(params)

    _, loss_value = sess.run([train_opt, loss],
                             feed_dict={
                                inputs: batch['inputs'],
                                seq_len: batch['seq_lengths'],
                                targets: ctc_utils.sparse_tuple_from(batch['targets']),
                                rnn_keep_prob: dropout,
                            })
    print ('Loss value at epoch ' + str(epoch) + ':' + str(loss_value))
    if epoch % 200 == 0:
      saver.save(sess,args.save_model,global_step=epoch)
    if epoch == 64000:
        # VALIDATION
        print ('Loss value at epoch ' + str(epoch) + ':' + str(loss_value))
        print ('Validating...')

        validation_batch, validation_size = primus.getValidation(params)

        val_idx = 0

        val_ed = 0
        val_len = 0
        val_count = 0

        while val_idx < validation_size:
            mini_batch_feed_dict = {
                inputs: validation_batch['inputs'][val_idx:val_idx+params['batch_size']],
                seq_len: validation_batch['seq_lengths'][val_idx:val_idx+params['batch_size']],
                rnn_keep_prob: 1.0
            }


            prediction = sess.run(decoded,
                                  mini_batch_feed_dict)

            str_predictions = ctc_utils.sparse_tensor_to_strs(prediction)


            for i in range(len(str_predictions)):
                ed = ctc_utils.edit_distance(str_predictions[i], validation_batch['targets'][val_idx+i])
                val_ed = val_ed + ed
                val_len = val_len + len(validation_batch['targets'][val_idx+i])
                val_count = val_count + 1

            val_idx = val_idx + params['batch_size']

        print ('[Epoch ' + str(epoch) + '] ' + str(1. * val_ed / val_count) + ' (' + str(100. * val_ed / val_len) + ' SER) from ' + str(val_count) + ' samples.')
        print ('Saving the model...')
        saver.save(sess,args.save_model,global_step=epoch)
        print ('------------------------------')

1.15.2
Training with 70880 and validating with 7875
<module 'ctc_model' from '/content/ctc_model.py'>


/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


串流輸出內容已截斷至最後 5000 行。
Loss value at epoch 17089:1.7898107
Loss value at epoch 17090:7.309721
Loss value at epoch 17091:2.7208114
Loss value at epoch 17092:3.95539
Loss value at epoch 17093:6.1070843
Loss value at epoch 17094:6.542411
Loss value at epoch 17095:5.473675
Loss value at epoch 17096:20.315435
Loss value at epoch 17097:8.519334
Loss value at epoch 17098:5.4467525
Loss value at epoch 17099:5.676798
Loss value at epoch 17100:9.191226
Loss value at epoch 17101:3.1126232
Loss value at epoch 17102:5.278857
Loss value at epoch 17103:4.271242
Loss value at epoch 17104:2.3290682
Loss value at epoch 17105:10.086662
Loss value at epoch 17106:9.390219
Loss value at epoch 17107:6.766654
Loss value at epoch 17108:9.135729
Loss value at epoch 17109:2.882468
Loss value at epoch 17110:3.424753
Loss value at epoch 17111:2.7401562
Loss value at epoch 17112:5.679969
Loss value at epoch 17113:2.4336355
Loss value at epoch 17114:3.1209002
Loss value at epoch 17115:2.284874
Loss value at epoch 1711

In [ ]:
from google.colab import files
files.download('/content/trained_semantic_model-6400.meta')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
for name in [6400]:
  files.download('/content/trained_semantic_model-'+str(name)+'.meta')
  files.download('/content/trained_semantic_model-'+str(name)+'.data-00000-of-00001')
  files.download('/content/trained_semantic_model-'+str(name)+'.index')
  files.download('/content/checkpoint')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>